In [1]:
import logging
import math
import random
from datetime import datetime
from enum import Enum
from pathlib import Path
from typing import List, Tuple

import pandas
import pyarrow.parquet as pq
import sentence_transformers.losses as losses
from datasets import Dataset
from sentence_transformers import (
    SentenceTransformer,
    SentenceTransformerTrainer,
    SentenceTransformerTrainingArguments,
    models
)
from sentence_transformers.evaluation import (
    EmbeddingSimilarityEvaluator,
    SimilarityFunction,
)
from sentence_transformers.readers import InputExample
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, TrainerCallback, TrainerControl, TrainerState

# Set up basic configuration for logging
logging.basicConfig(level=logging.INFO)


class QueryType(Enum):
    SHORT = "short_query"
    MEDIUM = "medium_query"
    LONG = "long_query"


def load_df(file: Path) -> pandas.DataFrame:
    loaded_table = pq.read_table(file)
    return loaded_table.to_pandas()


def convert_to_hf_dataset(dataframe: pandas.DataFrame, question_type:str) -> Dataset:
    # Convert each InputExample into a dictionary
    data_dict = {
        "anchor": [],
        "positive": [],
    }
    for _, row in dataframe.iterrows():
        data_dict['anchor'].append(row[question_type])
        data_dict['positive'].append(row['context'])
    # Create a Hugging Face Dataset
    return Dataset.from_dict(data_dict)


def get_train_and_eval_datasets(
    dataset_name: Path,
    question_type:str
) -> Tuple[Dataset, Dataset]:
    # NOTE francuzi su 70:15:15 ovde je 80:10:10
    df = load_df(file=dataset_name)
    train_df, eval_df = train_test_split(df, test_size=0.2, random_state=42)
    dataset_counts_train = train_df['dataset'].value_counts()
    dataset_counts_eval = eval_df['dataset'].value_counts()
    # Convert lists to Hugging Face Datasets
    train_dataset = convert_to_hf_dataset(train_df, question_type)
    eval_dataset = convert_to_hf_dataset(eval_df, question_type)

    return train_dataset, eval_dataset, dataset_counts_train, dataset_counts_eval



/home/selena/sele/SRBedding/.venv/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:11: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


## Testing

In [13]:
loaded_table = pq.read_table('datasets/wiki_4o.parquet')
df_wiki = loaded_table.to_pandas()

# loaded_table = pq.read_table('datasets/science.parquet')
# df_science = loaded_table.to_pandas()

loaded_table = pq.read_table('datasets/news_4o.parquet')
df_news = loaded_table.to_pandas()

loaded_table = pq.read_table('datasets/literature_4o.parquet')
df_literature = loaded_table.to_pandas()

# loaded_table = pq.read_table('datasets/wiki_4o.parquet')
# df_wiki_fixed = loaded_table.to_pandas()
loaded_table = pq.read_table('datasets/TRAIN11k_fixed_v2.parquet')
df_11k = loaded_table.to_pandas()


In [14]:
df_wiki

,context,short_query,medium_query,long_query,scores
0,UTF-8 UTF-8 varijanta je najzgodnija za kodira...,UTF-8 kodiranje,Unicode fontovi za prenosivost teksta,Kako koristiti UTF-8 u Microsoft Word-u?,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
1,Osnovna kodna strana na personalnim računarima...,Kodna strana PC437,Podrška za jezike kodne strane,Kako Unicode rešava problem kodnih strana?,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
2,Njen naziv je UCS-2 zato što koristi dva oktet...,UCS-2 karakteri,UTF-16 i problem prostora,Kako UTF-8 rešava problem prenosa podataka?,"{'long_query': 5, 'medium_query': 5, 'short_qu..."
3,Ova transformaciona šema je prevashodno zgodna...,transformaciona šema latinica,MIME standardi za prenos fajlova,Kako funkcioniše 7-bitna transformaciona šema ...,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
4,Osnovna razlika je u tome što UTF-7 koristi sa...,UTF-7 Base64 prednosti,UCS-4 Unicode standard karakteristike,Zašto je UCS-4 plan za budućnost Unicode-a?,"{'long_query': 4, 'medium_query': 4, 'short_qu..."
...,...,...,...,...,...
659,"Ova igra, u kojoj nema surovosti, veoma je zg...",pravila rekreativne košarke,pravila takmičarske košarke u NBA,koji su uslovi za takmičarsku košarku,"{'long_query': 5, 'medium_query': 5, 'short_qu..."
660,Većina terena ima parket izrađen od drveta Že...,visina obruča koša,dimenzije terena za košarku,razlika u liniji za 3 poena NBA i međunarodna,"{'long_query': 5, 'medium_query': 4, 'short_qu..."
661,Igrači su uniformisani i svi iz istog tima nos...,dresovi košarkaša,brojevi na dresovima košarkaša,koliko igrača sme biti na terenu,"{'long_query': 5, 'medium_query': 5, 'short_qu..."
662,"Zamene se mogu obavljati neograničeno, ali sam...",zamene sat zaustavljen,koliko traju četvrtine u NBA,koliko traju poluvremena na fakultetima,"{'long_query': 4, 'medium_query': 5, 'short_qu..."


In [15]:
df_wiki['dataset'] = 'wiki'
# df_wiki_fixed['dataset'] = 'wiki'
# df_science['dataset'] = 'science'
df_news['dataset'] = 'news'
df_literature['dataset'] = 'literature'

In [19]:
merged_df = pandas.concat([df_wiki, df_news, df_literature], ignore_index=True)
# Optionally, you can save the merged dataframe to a file
merged_df.to_parquet('datasets/TRAIN14o.parquet')

In [20]:
dataset_counts = merged_df['dataset'].value_counts()
print(dataset_counts)

dataset
wiki          664
news          636
literature    473
Name: count, dtype: int64


In [21]:
merged_df

,context,short_query,medium_query,long_query,scores,dataset
0,UTF-8 UTF-8 varijanta je najzgodnija za kodira...,UTF-8 kodiranje,Unicode fontovi za prenosivost teksta,Kako koristiti UTF-8 u Microsoft Word-u?,"{'long_query': 5, 'medium_query': 4, 'short_qu...",wiki
1,Osnovna kodna strana na personalnim računarima...,Kodna strana PC437,Podrška za jezike kodne strane,Kako Unicode rešava problem kodnih strana?,"{'long_query': 5, 'medium_query': 4, 'short_qu...",wiki
2,Njen naziv je UCS-2 zato što koristi dva oktet...,UCS-2 karakteri,UTF-16 i problem prostora,Kako UTF-8 rešava problem prenosa podataka?,"{'long_query': 5, 'medium_query': 5, 'short_qu...",wiki
3,Ova transformaciona šema je prevashodno zgodna...,transformaciona šema latinica,MIME standardi za prenos fajlova,Kako funkcioniše 7-bitna transformaciona šema ...,"{'long_query': 5, 'medium_query': 4, 'short_qu...",wiki
4,Osnovna razlika je u tome što UTF-7 koristi sa...,UTF-7 Base64 prednosti,UCS-4 Unicode standard karakteristike,Zašto je UCS-4 plan za budućnost Unicode-a?,"{'long_query': 4, 'medium_query': 4, 'short_qu...",wiki
...,...,...,...,...,...,...
1768,"To je opet meni na put stajalo, jer baš to sa...",sažalenje prama Branku,Laura spominje stare stvari,zašto je Fanika bila važna u priči?,"{'long_query': 3, 'medium_query': 4, 'short_qu...",literature
1769,"Već gotovo sam se i na Branka počeo jediti, z...",Branko i Laura,Zašto Fani mrzi Branka?,Kako se Branko zaljubio u Lauru umesto Ide?,"{'long_query': 5, 'medium_query': 5, 'short_qu...",literature
1770,"Pa sad Branko razvede takovu prediku, kakvu jo...",Branko i grofica,Branko i grofica razgovor o Idi,Kako je Branko reagovao na groficine suze?,"{'long_query': 5, 'medium_query': 5, 'short_qu...",literature
1771,Grofica je silno bogata; pet hiljada da nam d...,grofica Branku novac,zašto Branko ne želi tražiti novac,kako je Branko reagovao na predlog da traži novac,"{'long_query': 5, 'medium_query': 5, 'short_qu...",literature


In [22]:
tr, ev, dataset_counts_train, dataset_counts_eval = get_train_and_eval_datasets(dataset_name="datasets/TRAIN11k_fixed_v2.parquet", question_type="short_query")

In [23]:
dataset_proportions = dataset_counts_train / dataset_counts_train.sum()

# Print the proportions
print(dataset_proportions)

dataset_proportions = dataset_counts_eval / dataset_counts_eval.sum()

# Print the proportions
print(dataset_proportions)

dataset
wiki          0.632590
science       0.180130
news          0.150464
literature    0.036816
Name: count, dtype: float64
dataset
wiki          0.647034
science       0.173709
news          0.145113
literature    0.034144
Name: count, dtype: float64


In [24]:
tr

Dataset({
    features: ['anchor', 'positive'],
    num_rows: 9371
})

In [25]:
from pprint import pprint

pprint(tr['anchor'][2])
pprint(tr['positive'][2])


'Šta je uticalo na loše rezultate poslovanja?'
('319 stabilno poslovanje Drugi stratum čini takođe šest velikih privrednih '
 'subjekata koji su u istom vremenskom okviru imali narušenu finansijsku '
 'strukturu, što se direktno odrazilo i na loše rezutate poslovanja Na bazi '
 'predloženih kompanija formirane su dve polarizovane grupe velikih privrednih '
 'subjekata kako bi putem predloženog modela mogla da se pokaže homogenost u '
 'kretanju kvantitativnih i kvalitativnih pokazatelja Takođe, polarizovani '
 'pristup u istraživanju dao je mogućnost dodatnog testiranja pouzdanosti '
 'predloženog modela ocene kreditnog boniteta velikih privrednih subjekata')


In [30]:
ev

Dataset({
    features: ['sentence1', 'sentence2', 'score'],
    num_rows: 1128
})

In [35]:
from transformers import BertModel

# Load the model
model_name = "jerteh/Jerteh-355"  # Replace this with the correct model identifier
model = BertModel.from_pretrained(model_name)
model = model.half()
# Access the max sequence length
max_seq_length = model.config.max_position_embeddings

print(f"The maximum sequence length for the model is: {max_seq_length}")

You are using a model of type roberta to instantiate a model of type bert. This is not supported for all configurations of models and can yield errors.
Some weights of BertModel were not initialized from the model checkpoint at jerteh/Jerteh-355 and are newly initialized: ['embeddings.LayerNorm.bias', 'embeddings.LayerNorm.weight', 'embeddings.position_embeddings.weight', 'embeddings.token_type_embeddings.weight', 'embeddings.word_embeddings.weight', 'encoder.layer.0.attention.output.LayerNorm.bias', 'encoder.layer.0.attention.output.LayerNorm.weight', 'encoder.layer.0.attention.output.dense.bias', 'encoder.layer.0.attention.output.dense.weight', 'encoder.layer.0.attention.self.key.bias', 'encoder.layer.0.attention.self.key.weight', 'encoder.layer.0.attention.self.query.bias', 'encoder.layer.0.attention.self.query.weight', 'encoder.layer.0.attention.self.value.bias', 'encoder.layer.0.attention.self.value.weight', 'encoder.layer.0.intermediate.dense.bias', 'encoder.layer.0.intermediate.

The maximum sequence length for the model is: 514


In [2]:
loaded_table = pq.read_table('datasets/TRAIN11k_fixed_v2.parquet')
train = loaded_table.to_pandas()

In [5]:
for context in train.sample(5)['context']:
    print(context)

Harmonizacija definicija grada - ESPON 2013 Koncepti evropskih statističkih jedinica analize (NUTS; LAU), urbanih morfoloških područja funkcionalnih urbanih područja (FUA) prihvaćeni su, sinhronizovani i dalje razvijani u sklopu projekata ESPON 2013 Database (ESPON, 2013d; ESPON Database 2013, 2014; ESPON, 2013c) Evropske posmatračke mreže za teritorijalni razvoj i koheziju (ESPON - European Observation Network for Territorial Development and Cohesion) Baza podataka koja je nastala kao rezultat ovog projekta dalje se razvila kroz ESPON M4D (Multi Dimension Database Design and Development) projekat, koji za glavni cilj ima održavanje, ažuriranje, razvoj i proširenje baze podataka ESPON 2013 (ESPON, ESPON baza pruža temeljne regionalne informacije obezbjeđene iz ESPON-ovih projekata i od strane Eurostata, a koje se mogu koristiti kao podrška za teritorijalnu analizu razvoja na različitim geografskim nivoima
Od 1874. do 1882. godine, pohađao je gimnaziju. Za to vreme, napisao je svoje prv